

神经网络模仿人类神经系统，神经元，神经元之间的介质传递。神经网络的关键概念：神经元（包含权重、偏置）、层（一层包含多个神经元， 除输入、输出层外，其他称为隐藏层）、激活函数、前向传播、反向传播、epoch（一个前向、反向传播过程）、损失函数（计算一次正向传播与目标值之间的损失值，损失值越大表示差别越大）、优化器（一种降低损失函数结果的优化方法， 常见的有梯度下降（SGD、Momentum‌、NAG）、自适应学习率优化器（Adagrad‌、RMSprop‌、Adam‌、AdamW‌）、其他优化器（Muon‌）。

神经元包含对刺激（输入）的感知能力，产生应答（输出），其一般表示为f(x) = w(对刺激的感知权重)x+b(偏置)。
激活函数是一个非线性的函数。为什么需要是非线性的，因为假设激活函数是线性的，那对于神经元表示做激活函数可以得到f(x)=w2(w1x+b1)+b2 = w2w1x+w2b1+b2，结果表示依然线性，那多个线性层迭代的结果就是线性的（wx+b），整个神经网络就是一个函数，没有学习的能力。
神经网络单次训练经过神经元和激活函数到下一个神经元最后到输出的过程即为前向传播。降低损失函数优化神经元权重和偏置就是一个反向传播过程（如果没有反向传播，神经网络就没有学习能力）

FNN，前馈神经网络。是一种基础的神经网络。下面做一个简易实现：

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        # 输入层到隐藏层
        self.layer1 = nn.Linear(input_size, hidden_size)
        # 隐藏层到输出层
        self.layer2 = nn.Linear(hidden_size, output_size)
        # 激活函数 ReLU
        self.activation = nn.ReLU()
    # 前向传播
    def forward(self, x):
        # 输入层到隐藏层
        x = self.activation(self.layer1(x))
        # 隐藏层到输出层
        x = self.layer2(x)
        return x

# 创建模型实例
model = SimpleNN(input_size=10, hidden_size=20, output_size=2)

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
# 输入数据
inputs = torch.randn(10)
# 目标数据
targets = torch.randn(2)

# 训练循环（这里省略了数据加载部分）
for epoch in range(100):
    # 前向传播：传播过程中建设计算图，节点记录张量和运算函数，边记录张量关系
    outputs = model(inputs)
    # 计算损失
    loss = criterion(outputs, targets)
    # 梯度清零
    optimizer.zero_grad()
    # 反向传播：从loss张量开始，递归计算图自动微分
    loss.backward()
    # 更新参数
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/100], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 0.0862
Epoch [20/100], Loss: 0.0153
Epoch [30/100], Loss: 0.0010
Epoch [40/100], Loss: 0.0015
Epoch [50/100], Loss: 0.0012
Epoch [60/100], Loss: 0.0004
Epoch [70/100], Loss: 0.0001
Epoch [80/100], Loss: 0.0000
Epoch [90/100], Loss: 0.0000
Epoch [100/100], Loss: 0.0000


backforward简易实现

In [4]:
import numpy as np

class Tensor:
    def __init__(self, data, requires_grad=False, _children=(), _op=''):
        self.data = np.array(data)
        self.requires_grad = requires_grad
        self.grad = np.zeros_like(self.data) if requires_grad else None
        # 用于构建计算图
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op  # 操作名称，用于调试
        
    def __repr__(self):
        return f"Tensor({self.data}, requires_grad={self.requires_grad})"
    
    def __add__(self, other):
        other = other if isinstance(other, Tensor) else Tensor(other)
        out = Tensor(self.data + other.data, (self.requires_grad or other.requires_grad), (self, other), '+')
        
        def _backward():
            if self.requires_grad:
                # 加法操作的梯度传播：直接传递
                self.grad += out.grad
            if other.requires_grad:
                other.grad += out.grad
        out._backward = _backward
        
        return out
    
    def __mul__(self, other):
        other = other if isinstance(other, Tensor) else Tensor(other)
        out = Tensor(self.data * other.data, (self.requires_grad or other.requires_grad), (self, other), '*')
        
        def _backward():
            if self.requires_grad:
                # 乘法操作的梯度传播：乘以另一个操作数
                self.grad += other.data * out.grad
            if other.requires_grad:
                other.grad += self.data * out.grad
        out._backward = _backward
        
        return out
    
    def mean(self):
        out = Tensor(np.mean(self.data), self.requires_grad, (self,), 'mean')
        
        def _backward():
            if self.requires_grad:
                # 均值操作的梯度传播：均匀分布
                self.grad += np.ones_like(self.data) * out.grad / self.data.size
        out._backward = _backward
        
        return out
    
    def backward(self, gradient=None):
        # 拓扑排序：按计算顺序排列所有节点
        topo = []
        visited = set()
        
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        
        build_topo(self)
        
        # 初始化梯度
        if gradient is None:
            self.grad = np.ones_like(self.data)
        else:
            self.grad = gradient
            
        # 反向传播梯度
        for v in reversed(topo):
            v._backward()

# 测试示例
if __name__ == "__main__":
    # 创建叶子节点（模型参数）
    a = Tensor([2.0], requires_grad=True)
    b = Tensor([3.0], requires_grad=True)
    
    print("前向传播:")
    c = a * b  # 乘法操作
    d = c + 1  # 加法操作
    loss = d.mean()  # 均值操作
    
    print(f"a = {a}")
    print(f"b = {b}")
    print(f"c = a * b = {c}")
    print(f"d = c + 1 = {d}")
    print(f"loss = d.mean() = {loss}")
    
    print("\n反向传播前:")
    print(f"a.grad = {a.grad}")
    print(f"b.grad = {b.grad}")
    
    # 执行反向传播
    loss.backward()
    
    print("\n反向传播后:")
    print(f"a.grad = {a.grad}")  # 应该是 b * 1 = 3.0
    print(f"b.grad = {b.grad}")  # 应该是 a * 1 = 2.0
    
    # 验证结果
    print("\n手动验证:")
    print("∂loss/∂a = ∂loss/∂d * ∂d/∂c * ∂c/∂a")
    print("         = 1 * 1 * b =", b.data[0])
    print("∂loss/∂b = ∂loss/∂d * ∂d/∂c * ∂c/∂b")
    print("         = 1 * 1 * a =", a.data[0])

前向传播:
a = Tensor([2.], requires_grad=True)
b = Tensor([3.], requires_grad=True)
c = a * b = Tensor([6.], requires_grad=True)
d = c + 1 = Tensor([7.], requires_grad=True)
loss = d.mean() = Tensor(7.0, requires_grad=True)

反向传播前:
a.grad = [0.]
b.grad = [0.]

反向传播后:
a.grad = [3.]
b.grad = [2.]

手动验证:
∂loss/∂a = ∂loss/∂d * ∂d/∂c * ∂c/∂a
         = 1 * 1 * b = 3.0
∂loss/∂b = ∂loss/∂d * ∂d/∂c * ∂c/∂b
         = 1 * 1 * a = 2.0
